In [9]:
pip install pandas

You should consider upgrading via the '/Users/ajsmac/Desktop/PROJECTS/predict_stock_movement_based_on_news/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 KB 274.9 kB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/Users/ajsmac/Desktop/PROJECTS/predict_stock_movement_based_on_news/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install transformers

You should consider upgrading via the '/Users/ajsmac/Desktop/PROJECTS/predict_stock_movement_based_on_news/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install torch

You should consider upgrading via the '/Users/ajsmac/Desktop/PROJECTS/predict_stock_movement_based_on_news/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [12]:
url = "https://www.moneycontrol.com/news/tags/companies.html/page-1/"
resp = resp=requests.get(url)

In [13]:
soup = BeautifulSoup(resp.text)
script_tags = soup.find_all('script', type='application/ld+json')

for tag in script_tags:
    text = tag.text.strip()
    js = json.loads(text)

    if isinstance(js, dict) and js.get('@type', None) == 'ItemList':
        articles = js['itemListElement']
        articles = [{k: v for k, v in article.items() if k in ['name', 'url']} for article in articles]

In [14]:
data = pd.DataFrame(articles)
data.rename(columns={"name": "headline"}, inplace=True)

In [15]:
def get_article(url):
    resp = requests.get(url)
    
    soup = BeautifulSoup(resp.text)
    script_tags = soup.find_all('script', {'type': 'application/ld+json'})

    for tag in script_tags:
        text = tag.text.strip()
        js = json.loads(text, strict=False)
        if isinstance(js, list):
            
            article = js[0]
            if article.get('@type', None) == 'NewsArticle':
                date = article.get('datePublished', None)
                content = article.get('articleBody', None)
                return (date, content)

In [16]:
data[['date', 'content']] = data.apply(lambda row: pd.Series(get_article(row['url'])), axis=1)

In [17]:
data['date'] = pd.to_datetime(data['date'])
data = data[['date', 'headline', 'url', 'content']]

In [18]:
data.head()

,date,headline,url,content
0,2024-06-28 11:40:33+05:30,Ashok Leyland partners with Bajaj Finance for ...,https://www.moneycontrol.com/news/business/ash...,Commercial vehicle maker Ashok Leyland Ltd on ...
1,2024-06-28 10:53:57+05:30,UltraTech&rsquo;s stake buy in India Cements &...,https://www.moneycontrol.com/news/business/ult...,Highlights\r\n\r\n\r\n\tUltraTech has agreed t...
2,2024-06-27 13:37:22+05:30,Kajaria Ceramics &ndash; Solid execution shows...,https://www.moneycontrol.com/news/business/mon...,None
3,2024-06-27 13:12:22+05:30,Fairfax India Holdings unit sells 9.7% stake i...,https://www.moneycontrol.com/news/business/fai...,Fairfax India Holdings Corp&nbsp;said on Thurs...
4,2024-06-27 13:02:32+05:30,Indian Oil to boost LNG portfolio to 20 millio...,https://www.moneycontrol.com/news/business/ind...,Indian Oil aims to increase its liquified natu...


In [57]:
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          pipeline)
from torch import torch
classifier = pipeline("text-classification", model="ProsusAI/finbert", tokenizer="ProsusAI/finbert")

# Define the our_pipeline function
def our_pipeline(payload):
    result = [{"label":"none", "score":0}]
    if payload:
        result = classifier(payload[:512])
    return result


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [63]:
data['result'] = data['content'].apply(our_pipeline)

# Extract the 'label' and 'score' from the 'result' column
data['label'] = data['result'].apply(lambda x: x[0]['label'])
data['score'] = data['result'].apply(lambda x: x[0]['score'])

# Drop the 'result' column if you no longer need it
data = data.drop(columns=['result'])



In [78]:
import pandas as pd
from IPython.display import display

# Set the maximum column width to a large value
pd.set_option('max_colwidth', 1000)

# Display the DataFrame with the selected columns
display(data[["url", "label", "score"]])

,url,label,score
0,https://www.moneycontrol.com/news/business/ashok-leyland-partners-with-bajaj-finance-for-vehicle-finance-12758055.html,positive,0.935985
1,https://www.moneycontrol.com/news/business/ultratechs-stake-buy-in-india-cements-is-a-takeover-now-on-the-cards-12757968.html,positive,0.825984
2,https://www.moneycontrol.com/news/business/moneycontrol-research/kajaria-ceramics-solid-execution-shows-formidable-leadership-12757131.html,none,0.000000
3,https://www.moneycontrol.com/news/business/fairfax-india-holdings-unit-sells-9-7-stake-in-csb-bank-for-70-million-12757383.html,neutral,0.532279
4,https://www.moneycontrol.com/news/business/indian-oil-to-boost-lng-portfolio-to-20-million-tonnes-by-2030-executive-says-12757378.html,positive,0.940822
5,https://www.moneycontrol.com/news/business/air-india-to-deploy-a350-planes-on-delhi-london-route-from-september-1-12757334.html,positive,0.856334
6,https://www.moneycontrol.com/news/business/25-of-new-hires-are-married-women-nearly-70-of-workforce-are-women-foxconn-12757281.html,neutral,0.579447
7,https://www.moneycontrol.com/news/business/bpcl-sees-nearly-flat-annual-crude-processing-executive-says-12757277.html,negative,0.942877
8,https://www.moneycontrol.com/news/business/video-commerce-offerings-gain-traction-indians-spent-over-2-million-hours-video-shopping-flipkart-12757241.html,positive,0.694737
9,https://www.moneycontrol.com/news/business/infosys-ceo-parekh-pays-rs-25-lakh-to-sebi-for-failing-to-have-adequate-controls-to-prevent-insider-trading-12757190.html,negative,0.508210


In [59]:
data.to_csv('articles.csv')